# Notebook for selecting all comments ever in a particular set of subreddits

In [1]:
from pyspark.sql import SQLContext
import pandas as pd
from pyspark.sql.functions import *
import pyspark.sql

In [2]:
sqlC = SQLContext(sc)

In [3]:
reddit = sqlC.read.parquet("all_comments.parquet")

In [4]:
reddit.printSchema()

root
 |-- approved_at_utc: string (nullable = true)
 |-- approved_by: string (nullable = true)
 |-- archived: boolean (nullable = true)
 |-- author: string (nullable = true)
 |-- author_cakeday: boolean (nullable = true)
 |-- author_flair_css_class: string (nullable = true)
 |-- author_flair_text: string (nullable = true)
 |-- banned_at_utc: string (nullable = true)
 |-- banned_by: string (nullable = true)
 |-- body: string (nullable = true)
 |-- body_html: string (nullable = true)
 |-- can_gild: boolean (nullable = true)
 |-- can_mod_post: boolean (nullable = true)
 |-- collapsed: boolean (nullable = true)
 |-- collapsed_reason: string (nullable = true)
 |-- controversiality: long (nullable = true)
 |-- created: long (nullable = true)
 |-- created_utc: string (nullable = true)
 |-- distinguished: string (nullable = true)
 |-- downs: long (nullable = true)
 |-- edited: string (nullable = true)
 |-- gilded: long (nullable = true)
 |-- id: string (nullable = true)
 |-- is_submitter: bool

## Select just the columns we want to work with
- specifically, exclude columns that have sub-columns to them because they're a pain to work with if we don't need their data.

In [5]:
reddit = reddit[['approved_at_utc',  'approved_by', 'archived', 'author', 
                 'author_cakeday', 'author_flair_css_class', 'author_flair_text',
                 'banned_at_utc', 'banned_by',  'body',  'can_gild',
                 'can_mod_post', 'collapsed',  'collapsed_reason', 'controversiality',
                 'created', 'created_utc', 'distinguished', 'downs',  'edited',
                 'gilded',  'id', 'is_submitter', 'likes', 'link_id', 'name', 
                 'num_reports', 'parent_id', 'removal_reason', 'replies', 
                 'report_reasons', 'retrieved_on', 'saved', 'score', 'score_hidden',
                 'stickied', 'subreddit', 'subreddit_id', 'ups']]

## Clean up text
- Reddit lets users type almost anything. This can be a major pain down the road. To help, this code replaces all of the newlines, carriage returns, and tabs with spaces. 
    - yes, some information is lost this way. 

In [6]:
reddit = reddit.withColumn('body', regexp_replace('body', '\n', ' '))
reddit = reddit.withColumn('body', regexp_replace('body', '\r', ' '))
reddit = reddit.withColumn('body', regexp_replace('body', '\t', ' '))

## get a count of how many posts are in each subreddit
- this can help us decide which ones to download

In [7]:
tmp = reddit.groupby('subreddit').count()
tmp.show(10)

+-------------------+--------+
|          subreddit|   count|
+-------------------+--------+
|              anime|11324720|
|         MensRights| 3527064|
|         MLBTheShow|  762202|
|             travel| 1647254|
|UnresolvedMysteries|  655796|
|           arumba07|   23084|
|           lacrosse|  123522|
|         QuotesPorn|  339011|
|         ArtHistory|   25659|
|            ukraina|  673181|
+-------------------+--------+
only showing top 10 rows



## List subreddits of interest, see their post count
- case sensitive!

In [8]:
li = ['politics', 
      'Republican', 'republicans', #'askaconservative', 'CollegeRepublicans', 'askarepublican',
      'democrats', #'AskDemocrats', 'Democrat', 'CollegeDemocrats',
      'GreenParty',
      'Libertarian', #'AskLibertarians',
      'Liberal', #'AskALiberal', 
      'progressive',
      'socialism', #'AskASocialist',
      'demsocialist',
      'communism', 'FULLCOMMUNISM', #'communists', 'Communist', 
      #'votethirdparty',
      'NeutralPolitics'
     ]

tmp.filter(tmp.subreddit.isin(li)).sort('count', ascending=False).show(50)

+---------------+--------+
|      subreddit|   count|
+---------------+--------+
|       politics|61748124|
|    Libertarian| 3990229|
|      socialism| 1110657|
|  FULLCOMMUNISM|  494840|
|NeutralPolitics|  342213|
|     Republican|  333246|
|    progressive|  247961|
|      democrats|  172526|
|        Liberal|  152893|
|      communism|  123248|
|     GreenParty|   15215|
|    republicans|   11949|
|   demsocialist|    5428|
+---------------+--------+



In [9]:
len(li)

13

## Select and save the data from each subrddit separately
- First, filter (select where) the data so that we only have the subreddits we care about. Tell pyspark to cache that result for future reuse. This makes the rest of the searches after it faster.
    - pyspark actually uses lazy evaluation, so this doesn't run until the first call to `short_reddit.filter()`. That means the first loop is slow, but the loops after it go faster. 
- Save everything to a tsv file, one per subreddit
    - note the headers are off by default
    - note this actually saves to a folder full of tsv files (one for each partition). That's a spark thing good for spark but bad for other programs. We'll have to merge them outside this script. (There are ways to merge them here, but if the file is bigger than the JVM memory it throws errors. Easier to fix outside where that doesn't happen.)
- write settings
    - `mode('overwrite')` replaces the files if they exist already. Saves us from "file already exists" errors.
    - `header=True` is needed because by default CSVs don't have headers here.
    - `escapeQuotes=False` seems like a strange option, but with it off, the files get read in by pandas okay. With it on pandas gives parsing errors. 

In [10]:
print("making short...")
short_reddit = reddit.filter(reddit.subreddit.isin(li))
short_reddit.cache()

for l in li:
    print 'making', l
    result = short_reddit.filter(short_reddit.subreddit == l)
    result.write.mode('overwrite').csv(l+'.tsv.parts', sep='\t', 
                                       header=True, escapeQuotes=False)
print('done!')

making short...
making politics
making Republican
making republicans
making democrats
making GreenParty
making Libertarian
making Liberal
making progressive
making socialism
making demsocialist
making communism
making FULLCOMMUNISM
making NeutralPolitics
done!


## Same as above, but with a different list of subreddits

In [11]:
li = ['cats', #'CatsonGlass', 'CatLoaf', 'Kittens', 'Meow_irl',
      'CatGifs', 'StartledCats', #'StuffOnCats', 
      'CatsStandingUp',
      'dogs', 'dogpictures', #'dogswithjobs', 
      'Dogtraining', #'DOG', 
      'puppies'#, 'dogswearinghats'
     ]

tmp.filter(tmp.subreddit.isin(li)).sort('count', ascending=False).show(20)

+--------------+-------+
|     subreddit|  count|
+--------------+-------+
|          dogs|1504097|
|          cats|1479661|
|   Dogtraining| 319811|
|CatsStandingUp| 299892|
|   dogpictures| 187653|
|  StartledCats|  90970|
|       CatGifs|  23750|
|       puppies|  21052|
+--------------+-------+



In [12]:
print("making short...")
short_reddit = reddit.filter(reddit.subreddit.isin(li))
short_reddit.cache()

for l in li:
    print 'making', l
    result = short_reddit.filter(short_reddit.subreddit == l)
    result.write.mode('overwrite').csv(l+'.tsv.parts', sep='\t', 
                                       header=True, escapeQuotes=False)
    #df = result.toPandas()
    #df.to_csv(l+'.tsv', sep='\t', index=False)
print('done!')

making short...
making cats
making CatGifs
making StartledCats
making CatsStandingUp
making dogs
making dogpictures
making Dogtraining
making puppies
done!


In [13]:
len(li)

8

In [14]:
result = reddit.filter(reddit.subreddit == 'CatGifs')

In [15]:
result.write.mode('overwrite').csv('CatGifs.tsv', sep='\t', header=True, escapeQuotes=False)

In [16]:
print 'done!'

done!
